##Getting started

In [1]:
!git clone https://github.com/DionysiusSidorius/project-753.git

Cloning into 'project-753'...
remote: Enumerating objects: 601, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 601 (delta 75), reused 4 (delta 4), pack-reused 477 (from 1)
Receiving objects: 100% (601/601), 8.20 MiB | 6.22 MiB/s, done.
Resolving deltas: 100% (361/361), done.


In [ ]:
import pandas as pd
df = pd.read_csv('/content/project-753/spanish.csv', sep=None)

<ipython-input-2-d79e48a46742>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/content/project-753/spanish.csv', sep=None)


##Deleter

In [ ]:
with open('lexemes.txt', 'w') as lex:
  pass

##Lexemes

###nomina

In [ ]:
#NOUN(_V)
import re
dict_noun_V = df[
    (df['WORD'].str.strip().str.fullmatch(r".+[aoeiuáóéíú]\b")) & ((df['POS'].str.strip() == 'extranj.')|(df['POS'].str.strip() == 'sust.'))
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    if not isinstance(other, str):
          forms = set()
    else:
          forms = set([re.sub(r'(.+)s', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms and list(forms)[0] else ''}
 gramm: noun
 paradigm: NOM
"""
    )

In [ ]:
#NOUN(_C)
import re
dict_noun_C = df[
    (df['WORD'].str.strip().str.fullmatch(r'.*(?!.[aoeiuáóéíú]|os).{2}')) & ((df['POS'].str.strip() == 'extranj.')|(df['POS'].str.strip() == 'sust.'))
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_C['WORD'], dict_noun_C['ADDITIONAL_INFO']):
    forms = set([re.sub(r'(.+)es', r'\1', form) if word[-2:]!='es' else re.sub(r'(.+ess?)es', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms else ''}
 gramm: noun
 paradigm: NOM_C
"""
    )

In [ ]:
#NOUN(_os)
import re
dict_noun_os = df[
    (df['WORD'].str.strip().str.fullmatch(r'.+os\b')) & (df['POS'].str.strip() == 'sust.')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_os['WORD'], dict_noun_os['ADDITIONAL_INFO']):
    forms = set(eval(other)['Formas atestiguadas:\xa0'])
    forms.discard(word)
    if any([form[-2:]=='es' for form in forms]):
      forms = [re.sub(r'(.+)es', r'\1', form) for form in forms]
      lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms else ''}
 gramm: noun
 paradigm: NOM_C
""")
    else:
      lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms else ''}
 gramm: noun
 paradigm: NOM_os
"""
    )

In [ ]:
#ADJ(_V)
import re
dict_adj_V = df[
    (df['WORD'].str.strip().str.fullmatch(r".+o\b")) & (df['POS'].str.strip() == 'adj.')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_adj_V['WORD'], dict_adj_V['ADDITIONAL_INFO']):
    word = re.sub(r'(.+)(o)', r'\1', word)
    if not isinstance(other, str):
      forms = set()
    else:
      forms = set([re.sub(r'(.+)([oa]s?)', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word+'o'}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: adj
 paradigm: ADJ
"""
    )

In [ ]:
#ADJ(_e)
import re
dict_adj_e = df[
    (df['WORD'].str.strip().str.fullmatch(r".+e\b")) & (df['POS'].str.strip() == 'adj.')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_adj_e['WORD'], dict_adj_e['ADDITIONAL_INFO']):
    word = re.sub(r'(.+)(e)', r'\1', word)
    if not isinstance(other, str):
      forms = set()
    else:
      forms = set([re.sub(r'(.+)(es?)', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word+'e'}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: adj
 paradigm: ADJ_e
"""
  )

In [ ]:
#ADJ(_C/_Ca)
import re
dict_adj_C = df[
    (~df['WORD'].str.strip().str.contains(r'([aoeiuáóéíú]|os)$', regex=True, na=False)) & (df['POS'].str.strip() == 'adj.')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_adj_C['WORD'], dict_adj_C['ADDITIONAL_INFO']):
    if not isinstance(other, str):
      forms = set()
    else:
      forms = set([re.sub(r'(.+)(es|(?<=a)s)\b', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    if any([form[-1]=='a' for form in forms]):
      forms = set([re.sub(r'(.+)(a)\b', r'\1', form) for form in forms])
      forms.discard(word)
      lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: adj
 paradigm: ADJ_Ca
"""
    )
    else:
      forms.discard(word)
      lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: adj
 paradigm: ADJ_C
"""
    )

In [ ]:
#ADJ(_os) - son numerales (ADJ_V) o simplemente adjetivos (ADJ_C(a))
import re
dict_noun_os = df[
    (df['WORD'].str.strip().str.fullmatch(r'.+os\b')) & (df['POS'].str.strip() == 'adj.')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_os['WORD'], dict_noun_os['ADDITIONAL_INFO']):
    if re.fullmatch(r'.+ientos', word):
      word = re.sub(r'(.+)os\b', r'\1', word)
      forms = set([re.sub(r'(.+)[ao]s', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
      forms.discard(word)
      lex.write(
f"""-lexeme
 lex: {word+'os'}
 stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms else ''}
 gramm: num
 paradigm: ADJ
""")
    elif word=='dos':
      forms = set(eval(other)['Formas atestiguadas:\xa0'])
      forms.discard(word)
      lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms else ''}
 gramm: num
 paradigm: ZERO
""")
    else:
      forms = set(eval(other)['Formas atestiguadas:\xa0'])
      if any([re.fullmatch(r'.+as', form) for form in forms]):
        forms = set([re.sub(r'(.+)([oa]s)', r'\1', form) for form in forms])
        word = re.sub(r'(.+)os', r'\1', word)
        forms.discard(word)
        lex.write(
f"""-lexeme
 lex: {word+'os'}
 stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms else ''}
 gramm: adj
 paradigm: ADJ
"""
    )
      else:
        forms = set([re.sub(r'(.+)(es|(?<=a)s)\b', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
        forms.discard(word)
        lex.write(
f"""-lexeme
  lex: {word}
  stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms else ''}
  gramm: adj
  paradigm: ADJ_C
"""
      )

###Verba

In [ ]:
#VERB(_ar)
import re
dict_noun_V = df[
    (df['WORD'].str.strip().str.fullmatch(r".+ar\b")) &
     ((df['POS'].str.strip() == 'vrb.')|(df['POS'].str.strip() == 'Unidades pluriverbales:') &
    (
        (~df['WORD'].isin({'adar', 'atacir', 'azar', 'azingar', 'bezoar', 'cantar', 'cuer', 'haber', 'juglar', 'lugar', 'mar', 'menester', 'mujer', 'par', 'pesar', 'poder', 'saber', 'ser'}))
        | df.index.isin({5363, 7671, 7804, 8682})
    ))
]
tags = {'fut. indic. 2 pl.', 'fut. subj. 1 sing.', 'plusc. simp. 2 sing.', 'pot. simp. 1 sing.', 'imper. 2 pl.', 'fut. subj. 3 pl.', 'fut. subj. 2 pl.', 'pot. simp. 3 pl.', 'pret. 2 sing.', 'pres. indic. 1pl.', 'imperf. indic. 3 pl.', 'fut. perf. indic. 1 pl.', 'infinit. comp.', 'pret. anter. 3pl.', 'fut. subj. 3pl.', 'fut. perf. subj. 3 pl.', 'fut. subj. 3 sing.', 'p. p.', 'pret. perf. indic. 1 pl.', 'pres. subj. 3pl.', 'pret. perf. subj. 2 sing.', 'imperf. subj. 3 sing.', 'pret. plusc. indic. 3 pl.', 'fut. perf. subj. 2 pl.', 'fut. perf. subj. 1 pl.', 'pot. simp. 2 pl.', 'pret. perf. indic. 2 sing.', 'pret. plusc. subj. 3 sing.', 'imperf. indic. 1 sing.', 'ger. comp.', 'pres. subj. 1 pl.', 'pot. simp. 1pl.', 'plusc. si mp. 3 sing.', 'imperf. indic. 3pl.', 'fut. indic. 1pl.', 'pret. plusc. subj. 2 sing.', 'imperf. indic. 3 sing.', 'plusc. simp. 1 pl.', 'pres. indic. 3 sing.', 'pret. plusc. subj. 2 pl.', 'pres. subj. 1 sing.', 'pot. simp. 3 sing.', 'pret. perf. indic. 2 pl.', 'pres. indic. 1 pl.', 'pot. simp. 2 sing.', 'pret. 2 pl.', 'pret. perf. indic. 3 sing.', 'pret. perf. subj. 3 sing.', 'pot. simp. 3pl.', 'fut. perf. indic. 3 pl.', 'pret. anter. 3 pl.', 'plusc. simp. comp. 3 sing.', 'fut. perf. indic. 2 sing.', 'pres. subj. 2sing.', 'pret. plusc. indic. 3 sing.', 'plusc. simp. 2 pl.', 'pret. plusc. indic. 1 pl.', 'pres. subj. 1pl.', 'imperf. indic. 1 pl.', 'plusc. simp. comp. 3 pl.', 'pres. subj. 2 pl.', 'imper. 2pl.', 'fut. indic. 2pl.', 'pret. plusc. subj. 3pl.', 'pret. anter. 2 pl.', 'pot. simp. 1 pl.', 'fut. indic. 3pl.', 'pret. 2pl.', 'pres. subj. 3 sing.', 'pret. 1 sing.', 'fut. perf. subj. 3 sing.', 'pret. perf. indic. 3 pl.', 'plusc. simp. 1 sing.', 'pres. subj. 2pl.', 'infinit.', 'fut. indic. 3 pl.', 'imperf. indic. 2 sing.', 'pret. plusc. subj. 1 pl.', 'pret. anter. 3 sing.', 'pret. anter. 1 pl.', 'pres. indic. 3pl.', 'pret. perf. indic. 3pl.', 'imperf. subj. 1 sing.', 'plusc. simp. comp. 1 pl.', 'fut. indic. 3 sing.', 'pres. subj. 2 sing.', 'pres. subj. 3 pl.', 'fut. perf. subj. 2 sing.', 'ger.', 'imperf. subj. 3 pl.', 'fut. subj. 1 pl.', 'pres. indic. 1 sing.', 'plusc. simp. 3 sing.', 'pret. perf. indic. 1 sing.', 'pret. plusc. indic. 2 sing.', 'pret. perf. subj. 2 pl.', 'fut. indic. 2 sing.', 'fut. subj. 2 sing.', 'pret. 1pl.', 'pret. plusc. subj. 3 pl.', 'imperf. indic. 2pl.', 'imperf. subj. 2 sing.', 'pres. indic. 2pl.', 'pret. 3 pl.', 'imperf. subj. 2 pl.', 'imperf. indic. 2 pl.', 'pret. plusc. indic. 2 pl.', 'p. a.', 'plusc. simp. 3 pl.', 'fut. indic. 1 sing.', 'pot. comp. 3 pl.', 'pret. anter. 2 sing.', 'imper. 2 sing.', 'pret. perf. subj. 1 pl.', 'imperf. subj. 1pl.', 'pret. plusc. indic. 1 sing.', 'pret. perf. subj. 3 pl.', 'pres. indic. 2 pl.', 'plusc. simp. 3pl.', 'pres. indic. 2 sing.', 'pret. plusc. subj. 1 sing.', 'imperf. indic.', 'imperf. subj. 1 pl.', 'fut. indic. 1 pl.', 'pret. 3 sing.', 'pres. indic. 3 pl.', 'pret. perf. indic. 1pl.', 'pret. anter. 1 sing.', 'imperf. indic. 1pl.', 'pret. perf. indic. 2pl.', 'pret. 3pl.', 'pret. plusc. indic. 3pl.', 'pret. 1 pl.', 'imperf. subj. 3pl.'}
tatags = '|'.join(tags).replace('.', '\.')
# clas = set()
pattern = r'(o|((a|e)((r(\(?[smtnl]?(e|a|o)?s?\)s?)?i?(a|e)|(b|u)a|ss?e)?(s(t(es?)?)?|mos|des|(ro)?n)?|l\(?l(e|a|o)?s?\)s?|r(\(?[smtn]?(e|a|o)?s?\)s?)?|t|ndo|(d([ao]s?)?)?)))$'
with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    if not isinstance(other, str):
      forms = set()
    else:
      forms = set(eval(other)['Formas atestiguadas:\xa0'] if other!= '{}' else [])
      wirds = set()
      for form in forms:
        for subs in form.split(';'):
          two = subs.split('–')
          if len(two)==2:
            # clas.add(two[0].strip())
            wirds.add(re.sub(tatags, '', two[1]).strip())
          else: wirds.add(re.sub(tatags, '', two[0]).strip())
      #forms = set([re.sub(r'(.+)([oa]s?)', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    wirds = set(re.sub(pattern, '', p) for p in wirds if p.count(' ')==0 and p.count('(')==0 and p.count(')')==0)
    wirds.discard(word[:-2])
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word[:-2]}.{'|'+'|'.join([form+"." for form in wirds]) if wirds and list(wirds)[0] else ''}
 gramm: verb
 paradigm: V_{word[-2]}
"""
    )
    # lex.write(f"{word}, ':', {wirds}\n")

In [ ]:
import re
pattern = r'(o|((a|e)((r(\(?[smtnl]?(e|a|o)?s?\)s?)?i?(a|e)|(b|u)a|ss?e)?(s(t(es?)?)?|mos|des|(ro)?n)?|l\(?l(e|a|o)?s?\)s?|r(\(?[smtn]?(e|a|o)?s?\)s?)?|t|ndo|(d([ao]s?)?)?)))$'
a = {'peno', 'peneste', 'penando', 'penaua', 'penan', 'penes', 'penassen', 'penades', 'penauan', 'penado', 'penados', 'penasse', 'penar', 'penarie', 'penare', 'penaron', 'penamos', 'penara', 'penadas', 'penaredes', 'penad'}
a = {'amostro', 'muestra', 'mostrasse', 'amostrar(l)an', 'mostrastes', 'mostreste', 'mostrad', 'mostrara', 'muestran', 'muestren', 'amostrasse', 'mostrassen', 'mostrat', 'mostrar(s)a', 'mostradas', 'mostrarie', 'muestras', 'muestre', 'mostresse', 'amostrare', 'mostre', 'mostremos', 'mostrest', 'mostraria', 'amostrarien', 'mostrare', 'mostrados', 'mostro', 'mostraremos', 'mostral(le)', 'amuestren', 'mostrauan', 'mostra', 'mostramos', 'muestres', 'amostren', 'mostraua', 'amostrad', 'mostrando', 'amuestra', 'amostre', 'amostramos', 'mostredes', 'mostrado', 'mostraran', 'mostraredes', 'mostar', 'amuestras', 'mostraren', 'amuestran', 'mostraron', 'amostrado', 'mostrarien', 'mostrada', 'amostradas', 'amostra', 'mostrar', 'mostrades', 'amostrar', 'amostrados', 'amostrada', 'mostraras', 'mostrares'}
a = {'enuiaron', 'ouo enuiado', 'embiarie', 'enuiaran', 'en uiasse', 'enbiado', 'enuia', 'enbie', 'enuiados', 'enuiase', 'enuiades', 'enuiauan', 'enbiassen', 'embiado', 'embio', 'enuian', 'enuiasses', 'enuiest', 'enujaua', 'enuiassemos', 'enuies', 'enuiar (las) as', 'enbiasse', 'enbiastes', 'auia enbiada', 'enbiaria', 'enuio', 'enbiare', 'enuiara', 'enbiarie', 'enbjasse', 'enuiat', 'enuiar', 'enuiast', 'auie enbiados', 'enbiara', 'enuiando', 'enuiaramos', 'enuiassen', 'ouo enbiada', 'enuiamos', 'enuiedes', 'enuiemos', 'enbiemos', 'enbiaron', 'enuiaries', 'enbies', 'enbiad', 'ouieren enuiado', 'embiaron', 'enuiaras', 'enuiadas', 'enujauan', 'enbieste', 'enujo', 'enuie', 'enuiaredes', 'embiar', 'enuiar (me l)edes', 'enbiades', 'enuiar (la) a', 'enbiase', 'en uiado', 'enuiaua', 'auia enbiado', 'enbiados', 'enuiare', 'enviado', 'en uiaron', 'enuiarie', 'enbiaran', 'enbiada', 'enuiarien', 'enuiaremos', 'enuiada', 'enuiaste', 'ouieron enuiadas', 'enuiastes', 'enbiauan', 'embie', 'ouiesse enuiada', 'enuiasse', 'enuiado as', 'enuien', 'enbiaua', 'embia', 'enujados', 'auien enuiado', 'enuias', 'embiastes', 'enuiares', 'enuiaren', 'emuio', 'enuiado', 'enbiar', 'enbiedes', 'emuiar', 'embian', 'embiaua', 'enbiarien', 'enbio', 'enuiad', 'auie enuiado', 'enuieste', 'enbia'}
a = {'escandalizan', 'escandalizemos', 'escandalizando', 'escandalizarien', 'escandalizar', 'escandalizados', 'escandalizasse', 'escandalizassen', 'escandalizado', 'escandalizen'}

# a = {'amal(le)'}
a = set(re.sub(pattern, '', p) for p in a if p.count(' ')==0)
print(a)

In [ ]:
#VERB(_er/ir)
import re
dict_noun_V = df[
    (df['WORD'].str.strip().str.fullmatch(r".+[ei]r\b")) &
          ((df['POS'].str.strip() == 'vrb.')|(df['POS'].str.strip() == 'Unidades pluriverbales:') &
    (
        (~df['WORD'].isin({'adar', 'atacir', 'azar', 'azingar', 'bezoar', 'cantar', 'cuer', 'haber', 'juglar', 'lugar', 'mar', 'menester', 'mujer', 'par', 'pesar', 'poder', 'saber', 'ser'}))
        | (df.index.isin({5363, 7671, 7804, 7682, 8346}))
    ))
]
tags = {'fut. indic. 2 pl.', 'fut. subj. 1 sing.', 'plusc. simp. 2 sing.', 'pot. simp. 1 sing.', 'imper. 2 pl.', 'fut. subj. 3 pl.', 'fut. subj. 2 pl.', 'pot. simp. 3 pl.', 'pret. 2 sing.', 'pres. indic. 1pl.', 'imperf. indic. 3 pl.', 'fut. perf. indic. 1 pl.', 'infinit. comp.', 'pret. anter. 3pl.', 'fut. subj. 3pl.', 'fut. perf. subj. 3 pl.', 'fut. subj. 3 sing.', 'p. p.', 'pret. perf. indic. 1 pl.', 'pres. subj. 3pl.', 'pret. perf. subj. 2 sing.', 'imperf. subj. 3 sing.', 'pret. plusc. indic. 3 pl.', 'fut. perf. subj. 2 pl.', 'fut. perf. subj. 1 pl.', 'pot. simp. 2 pl.', 'pret. perf. indic. 2 sing.', 'pret. plusc. subj. 3 sing.', 'imperf. indic. 1 sing.', 'ger. comp.', 'pres. subj. 1 pl.', 'pot. simp. 1pl.', 'plusc. si mp. 3 sing.', 'imperf. indic. 3pl.', 'fut. indic. 1pl.', 'pret. plusc. subj. 2 sing.', 'imperf. indic. 3 sing.', 'plusc. simp. 1 pl.', 'pres. indic. 3 sing.', 'pret. plusc. subj. 2 pl.', 'pres. subj. 1 sing.', 'pot. simp. 3 sing.', 'pret. perf. indic. 2 pl.', 'pres. indic. 1 pl.', 'pot. simp. 2 sing.', 'pret. 2 pl.', 'pret. perf. indic. 3 sing.', 'pret. perf. subj. 3 sing.', 'pot. simp. 3pl.', 'fut. perf. indic. 3 pl.', 'pret. anter. 3 pl.', 'plusc. simp. comp. 3 sing.', 'fut. perf. indic. 2 sing.', 'pres. subj. 2sing.', 'pret. plusc. indic. 3 sing.', 'plusc. simp. 2 pl.', 'pret. plusc. indic. 1 pl.', 'pres. subj. 1pl.', 'imperf. indic. 1 pl.', 'plusc. simp. comp. 3 pl.', 'pres. subj. 2 pl.', 'imper. 2pl.', 'fut. indic. 2pl.', 'pret. plusc. subj. 3pl.', 'pret. anter. 2 pl.', 'pot. simp. 1 pl.', 'fut. indic. 3pl.', 'pret. 2pl.', 'pres. subj. 3 sing.', 'pret. 1 sing.', 'fut. perf. subj. 3 sing.', 'pret. perf. indic. 3 pl.', 'plusc. simp. 1 sing.', 'pres. subj. 2pl.', 'infinit.', 'fut. indic. 3 pl.', 'imperf. indic. 2 sing.', 'pret. plusc. subj. 1 pl.', 'pret. anter. 3 sing.', 'pret. anter. 1 pl.', 'pres. indic. 3pl.', 'pret. perf. indic. 3pl.', 'imperf. subj. 1 sing.', 'plusc. simp. comp. 1 pl.', 'fut. indic. 3 sing.', 'pres. subj. 2 sing.', 'pres. subj. 3 pl.', 'fut. perf. subj. 2 sing.', 'ger.', 'imperf. subj. 3 pl.', 'fut. subj. 1 pl.', 'pres. indic. 1 sing.', 'plusc. simp. 3 sing.', 'pret. perf. indic. 1 sing.', 'pret. plusc. indic. 2 sing.', 'pret. perf. subj. 2 pl.', 'fut. indic. 2 sing.', 'fut. subj. 2 sing.', 'pret. 1pl.', 'pret. plusc. subj. 3 pl.', 'imperf. indic. 2pl.', 'imperf. subj. 2 sing.', 'pres. indic. 2pl.', 'pret. 3 pl.', 'imperf. subj. 2 pl.', 'imperf. indic. 2 pl.', 'pret. plusc. indic. 2 pl.', 'p. a.', 'plusc. simp. 3 pl.', 'fut. indic. 1 sing.', 'pot. comp. 3 pl.', 'pret. anter. 2 sing.', 'imper. 2 sing.', 'pret. perf. subj. 1 pl.', 'imperf. subj. 1pl.', 'pret. plusc. indic. 1 sing.', 'pret. perf. subj. 3 pl.', 'pres. indic. 2 pl.', 'plusc. simp. 3pl.', 'pres. indic. 2 sing.', 'pret. plusc. subj. 1 sing.', 'imperf. indic.', 'imperf. subj. 1 pl.', 'fut. indic. 1 pl.', 'pret. 3 sing.', 'pres. indic. 3 pl.', 'pret. perf. indic. 1pl.', 'pret. anter. 1 sing.', 'imperf. indic. 1pl.', 'pret. perf. indic. 2pl.', 'pret. 3pl.', 'pret. plusc. indic. 3pl.', 'pret. 1 pl.', 'imperf. subj. 3pl.'}
tatags = '|'.join(tags).replace('.', '\.')
# clas = set()
pattern = r'(os?|[ae](ron|mos|d(es)?|s|n|t|r|ss?[ei](s|mos|des|n)?)?|(y|i|j)(ss?[ei](s|mos|des|n)?|d|t|mos|des|d[oa]s?|r(on)?|e?st(es?)?|o([ls][eoa]?s?)?|[ae](ron|s|mos|des|n|ndo|r([ea](s|mos|des|n)?)?|ss?[ei](s|mos|des|n)?)?)?|[ie]?ri?[ae](s|mos|des|n)?|ud[oa]s?|t[oa]s?)$'
with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    if not isinstance(other, str):
      forms = set()
    else:
      forms = set(eval(other)['Formas atestiguadas:\xa0'] if other!= '{}' else [])
      wirds = set()
      for form in forms:
        for subs in form.split(';'):
          two = subs.split('–')
          if len(two)==2:
            # clas.add(two[0].strip())
            wirds.add(re.sub(tatags, '', two[1]).strip())
          else: wirds.add(re.sub(tatags, '', two[0]).strip())
      #forms = set([re.sub(r'(.+)([oa]s?)', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    wirds = set(re.sub(pattern, '', p) for p in wirds if p.count(' ')==0 and p.count('(')==0 and p.count(')')==0)
    wirds.discard(word[:-2])
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word[:-2]}.{'|'+'|'.join([form+"." for form in wirds if form]) if wirds and list(wirds)[0] else ''}
 gramm: verb
 paradigm: V_{word[-2]}
"""
    )
    # lex.write(f"{word}, ':', {wirds}\n")

In [ ]:
import re
# o|[ae](mos|d(es)?|s|n|t|r)?|(y|i)(mos|des|d[oa]s?|r|ist(es?)?|o|a|e(ron|s|mos|des|n|endo|r[ea](s|mos|des|n)?|ss?e(s|mos|des|n)?)?)?|[ie]?ri?[ae](s|mos|des|n)?
pattern = r'(o|[ae](ron|mos|d(es)?|s|n|t|r|ss?[ei](s|mos|des|n)?)?|(y|i|j)(ss?[ei](s|mos|des|n)?|d|t|mos|des|d[oa]s?|r(on)?|e?st(es?)?|o([ls][eoa]?s?)?|[ae](ron|s|mos|des|n|ndo|r([ea](s|mos|des|n)?)?|ss?[ei](s|mos|des|n)?)?)?|[ie]?ri?[ae](s|mos|des|n)?|ud[oa]s?|t[oa]s?)$'
a = {'ouiera contesçido', 'auie contesçudo', 'contesciendo', 'contesçe', 'contecer', 'contesçieran', 'contecera', 'contecie', 'contescio', 'contesciere', 'conteçudo', 'contesçudo', 'contesçio', 'conteciesse', 'contece', 'conteciera', 'contecer (l)a', 'contesçieron', 'contecieron', 'contescie', 'auie contecido', 'contescen', 'contesçra', 'contescido', 'contescran', 'conteçe', 'conteçie', 'contezcan', 'contesçiera', 'contescer', 'contecido', 'ouiesse contescido', 'contesçiesse', 'conteçen', 'contençer', 'conteçio', 'auie contescida', 'conteçrie', 'contesciol', 'conteçido', 'contesçrie', 'contecen', 'contesca', 'contecio', 'contesciesse', 'conteçra', 'conteciessen', 'contescien', 'conteçeran', 'a conteçudo', 'contesciessen', 'a contescido', 'contescida', 'contesciera', 'contescieran', 'contesce', 'contescra', 'conteçiera', 'contescrie', 'ayan contescido', 'auie conteçido', 'contescieron', 'contesçido'}
a = {'a caesçio', 'acahesciesse', 'acaesçer (les) an', 'acayecen', 'acaescer (te) a', 'acaece', 'ouiesse acaecido', 'acaesçieran', 'accaecieren', 'acaese', 'acaeçe', 'accaeçcer', 'achaecera', 'acaecier', 'acaesciera', 'accaecer(l)an', 'acaescera', 'acaescio', 'accaeçeran', 'accaecyendo', 'acaeçio', 'accaeciendo', 'acaesçen', 'acaescer (les) an', 'accaeziendo', 'acaeçier', 'acaecyere', 'accaecyere', 'acaesçe', 'accaecer (les) an', 'acaeçieron', 'acaeçca', 'accaecen', 'acaecieren', 'acaesciendo', 'acaeçra', 'accaescera', 'auie acaecido', 'acaeçer (les) an', 'acaeçera', 'acaescrie', 'acaecio', 'acaescier', 'accaeçe', 'acaescen', 'acaesciessi', 'acaesçieren', 'ouiesse acaescido', 'acaescran', 'acahesçieron', 'acahescio', 'accaecyeren', 'acaesçrie', 'a caescie', 'accaecera', 'acaesçiera', 'acaesçiesse', 'acaesçido', 'acaecer(l)a', 'acaesçien', 'acaeçiere', 'acaecel(les) ye', 'accaeçran', 'accaeçen', 'acahesçien', 'acaescia', 'acaesçieron', 'acaescieren', 'acaeciera', 'a acaescido', 'acaesçerie', 'acaezen', 'acahescen', 'acahesciessen', 'acaescido', 'a caescio', 'auien acaescido', 'accaeçcan', 'acaesce', 'acahescieron', 'acahesciera', 'acaeçran', 'acaesciesse', 'acaesçio', 'accacieren', 'accaecyo', 'acahescien', 'acaeciesse', 'acaescieran', 'acaeze', 'acaeçieren', 'acaescer', 'accaece', 'acahesçe', 'accaecere', 'acaescidos', 'acaescieron', 'acaescien', 'accaecio', 'acaecer(l)an', 'acaescer(l)a', 'accaeçra', 'acaecie', 'acaesca', 'acaecera', 'accaeceran', 'a caesciesse', 'auien acaescidos', 'acaesçra', 'acaeciere', 'acaezca', 'acahescie', 'acaeçen', 'acaesçiere', 'accaecer (lis) an', 'accaecer', 'accaeçera', 'acaecen', 'acaeciessen', 'accaesce', 'acaezcan', 'acaesçer', 'accaecer (l)a', 'acahescer', 'acaesciessen', 'acaeceran', 'acaecieran', 'acaecieron', 'acaeçer', 'acaesçier', 'acaesciere', 'acaescie', 'acaeciendo', 'acaesçie', 'acaecer', 'a caecier', 'accaeciere'}
a= {'aduzria', 'adugades', 'aduzre', 'aduzir (los) he', 'aduzrien', 'adux', 'a dozir', 'aduzir (l)e', 'aduxiemos', 'adugamos', 'aduchas', 'adozimos', 'adurie', 'aduzra', 'aduzriedes', 'adu', 'aduxesse', 'aduxieremos', 'adugan', 'aduxiste', 'aduzia', 'adozid', 'auie aducho', 'aduga', 'adugo', 'adugas', 'a duze', 'aduxieren', 'aduzir', 'a duxo', 'aduzie', 'adoxieron', 'aduxiera', 'adozie', 'aduzras', 'adozit', 'adozides', 'aduzes', 'adozia', 'aducho', 'adozir (los) emos', 'aduxier', 'aduxies', 'aduxien', 'aduzen', 'adozrie', 'aduziendo', 'aducha', 'a dozir (lo) as', 'aduze', 'aduzid', 'aduxiessen', 'aduzien', 'aduxiessemos', 'aduzrie', 'aduxiere', 'aduz', 'a duxiste', 'aduzimos', 'auemos aduchas', 'aduchos', 'aduxieran', 'aduxieron', 'aduziedes', 'adura', 'aduzran', 'aduxiestes', 'auedes aducho', 'adozir', 'auie aduchas', 'aduxeron', 'aduxiesse', 'aduremos', 'aduran', 'aduxist', 'aduxo'}
a = {'aprendieron', 'aprendemos', 'apprendieron', 'aprendan', 'aprendas', 'aprendie', 'aprendudas', 'aprisiera', 'aprendet', 'aprisieren', 'aprendien', 'auien aprendido', 'apris', 'apriso', 'aprended', 'aprendiesse', 'apprendra', 'auia apreso', 'auie apreso', 'aprendiste', 'aprendades', 'aprendrien', 'aprendiessen', 'aprisiessen', 'aprende', 'aprendi', 'a prendie', 'aprendran', 'aprend', 'apprender', 'aprendio', 'aprendiedes', 'apprendas', 'a prendieredes', 'auie aprendudo', 'e aprendudo', 'aprendist', 'a prisiessen', 'apresiere', 'aprendo', 'aprendiemos', 'auiendo aprendudo', 'aprendremos', 'aprendiendo', 'aprisiesse', 'ouo aprendudas', 'aprisieran', 'aprendudo', 'aprendiera', 'aprisiestes', 'aprendimos', 'aprendido', 'ouo apreso', 'aprender', 'aprendieran', 'aprenden', 'aprenda', 'apreso', 'aprisieron', 'ouiesse apreso'}
a = {'comjen', 'ouiesse comido', 'comiemos', 'comimos', 'comas', 'comiesse', 'comjeron', 'comieran', 'ouieres comido', 'comeras', 'comiessedes', 'combredes', 'combran', 'combras', 'comer', 'coman', 'comerien', 'comudos', 'comidas', 'comudo', 'comiere', 'comieron', 'comiera', 'comistes', 'aurien comido', 'combre', 'comidos', 'come', 'comieredes', 'comia', 'comiendo', 'ouieron comidos', 'conbran', 'a comuda', 'aya comido', 'auien comida', 'comi', 'combrie', 'comieres', 'comiestes', 'ouieron comido', 'comier', 'comida', 'comen', 'comel(lo)', 'comed', 'comie', 'comedes', 'a comido', 'comamos', 'comades', 'comido', 'comio', 'comien', 'comj', 'auie comido', 'comiessen', 'comet', 'comuda', 'a comudo', 'combremos', 'comjo', 'comjendo', 'comeran', 'conbrien', 'comjessen', 'coma', 'comieren', 'ouiemos comido', 'combrien', 'combra', 'ouo comida'}
a = {'connosçiendo', 'connosçe', 'connoçran', 'connosçidos', 'connozca', 'connoscieron', 'connosçudo', 'connoscieren', 'connoscrien', 'connoçudas', 'connoz', 'ouiesse conosçido', 'connosciesse', 'connosciendo', 'conosçido', 'connosciste', 'conosçiessen', 'connosçien', 'connoscel(le)', 'conosçuda', 'conosca', 'connocer', 'connocieren', 'connosçredes', 'connoscio', 'conoscer', 'connoscremos', 'connosçudos', 'connoscedes', 'connoscemos', 'conoçudo', 'connosçrie', 'connocida', 'connosçieron', 'connosçre', 'connoscas', 'connosçran', 'connoçien', 'conosçieron', 'cognoscan', 'connosciemos', 'conosçer', 'connusçuda', 'conocidas', 'conosçio', 'connosciere', 'connoscien', 'connosçed', 'connocidas', 'connoçudo', 'connocen', 'conosçudo', 'conoscen', 'connosciera', 'conoscia', 'connoscan', 'connoçedes', 'connoscimos', 'connoscist', 'connoscudo', 'connosco', 'connosces', 'connocio', 'conoscio', 'connoscades', 'connosci', 'connocien', 'connoçuda', 'auie connosçudos', 'connocj', 'connociera', 'connosce', 'connoce', 'conosçien', 'connoscido', 'connosçio', 'connoscidas', 'connosciessen', 'connosced', 'connosçieran', 'connoceras', 'conoscan', 'connosçudas', 'connocieron', 'connoscie', 'connoscamos', 'connosca', 'conocer', 'conosco', 'ouiesse connosçudo', 'connosçer', 'connoçen', 'connoscida', 'conosçie', 'connocieres', 'ouieron conosçudas', 'connocie', 'connoscieres', 'connoscieran', 'connoçer', 'connosçuda', 'conosçidos', 'connoscudas', 'connoscudos', 'conosçudos', 'conosçia', 'conosciessen', 'connoscen', 'connozcas', 'ouiessen connosçudo', 'connociessen', 'connoscer', 'conoscieron', 'connociendo', 'connosçra', 'conosciesse', 'connosçiessen', 'connocer(l)ien', 'conosçudas', 'connoçras', 'conosçieren', 'connoscidos', 'connoscra'}
a = {'conquiriessen', 'conquiriera', 'conquiridas', 'conquerira', 'conquirieron', 'con querir', 'auedes conquerido', 'conquerie', 'conquerimos', 'conquirrie', 'conquerire', 'auien conquerida', 'ouo conquerido', 'conquerido auie', 'conquiridos', 'auie conquiridos', 'conqueririen', 'conquerir (los) an', 'conquerre', 'conquiramos', 'conquiriesses', 'ouieron conquiridas', 'conqueriendo', 'conquerieron', 'ouiessen conquerida', 'conquerra', 'conquiriron', 'conqueriste', 'conquiriran', 'conqueril(los)', 'conquieren', 'conqueridas', 'conquerio', 'conquiri', 'conquerrie', 'conquirimos', 'conquiriesse', 'conquirieran', 'conquirir', 'auemos conquerido', 'conquerisse', 'conqueriran', 'conquerist', 'conquirio', 'conquerida', 'conquirida', 'conquirien', 'ouo conquerida', 'conquerido', 'ouiera conqueridas', 'conquiriendo', 'conquerir (las) a', 'conquerran', 'conquiriere', 'conquieres', 'conquirie', 'ouiese conquerida', 'conqueri', 'conqujrio', 'conqueriron', 'auiendo conqueridas', 'conquerir'}
a = {'cresçer', 'cresciendo', 'creçidos', 'cresceras', 'creçier', 'crecie', 'cresced', 'crescieredes', 'crecidas', 'creciendo', 'cresçiesse', 'creceras', 'cresçudo', 'crescera', 'cresçido', 'creçeras', 'crescien', 'creçen', 'crescieran', 'crecera', 'crecieran', 'crecier', 'crecer', 'cresca', 'crecen', 'crecio', 'creçra', 'creciessen', 'creçiendo', 'ouo crescido', 'creced', 'crescas', 'crescet', 'creceran', 'creciesse', 'crescer', 'cresce', 'crescra', 'crecieren', 'cresçieran', 'gresçiera', 'cresçer (les) a', 'creçer (l)a', 'creçet', 'creçie', 'auien cresçudo', 'cresçieron', 'cresçe', 'crescer (nos) a', 'cresceran', 'creçio', 'crescieron', 'crescie', 'cresçrie', 'creçiera', 'cresçiendo', 'crescio', 'creçer', 'cresçie', 'creciere', 'crezca', 'crescido', 'cresciesse', 'creçe', 'crezcan', 'crescida', 'crecieron', 'creçiessen', 'crecries', 'creçran', 'crescan', 'crescrie', 'cresçran', 'crescen', 'crecien', 'crecer (gela) an', 'cresçen', 'crecidos', 'cresçida', 'cresciessen', 'crescrien', 'cresçio', 'cresciera', 'crece'}
a = {'cubriesse', 'crubiol', 'crobi', 'crubas', 'crubiron', 'crobiesse', 'cobriran', 'cubre', 'cubririen', 'cubiertas', 'cruben', 'cubiertos', 'cubriras', 'cubrieron', 'cubra', 'cubriessen', 'cubriendo', 'crubiendo', 'cubren', 'cubrio', 'crubie', 'cruba', 'cobrir', 'crubrieron', 'crubiera', 'crube', 'cobrieron', 'cobierto', 'crubire', 'cubierto', 'crobiron', 'cobiertos', 'crubisse', 'cubririe', 'crubieron', 'crobisen', 'crubist', 'crobieran', 'cobrira', 'cubran', 'crubiesse', 'cobrieres', 'cubrie', 'crobira', 'crubien', 'cubras', 'crubrio', 'cubrir', 'crobir', 'crobistes', 'cubierta', 'cubrien', 'crubio', 'crubiran', 'cobriesse', 'crobieron', 'crubiedes', 'crobrir (te) a', 'crubririe'}
a = {'conplieron', 'cumplamos', 'complir (la) as', 'complido', 'compliran', 'cumpliera', 'conplira', 'cumplio', 'compliere', 'cumplidas', 'compliremos', 'cumplas', 'conplir', 'compliron', 'complieres', 'ouiesse cumplido', 'cunplieron', 'cumplen', 'cumplirie', 'cumplires', 'compliras', 'cunplan', 'cumplisse', 'complissen', 'cumpliste', 'ouiere complida', 'cumplir (te lo) he', 'compliessen', 'complidas', 'ouissen complido', 'complieron', 'cumpliendo', 'cumplida', 'cumpliessen', 'conplidas', 'cumplira', 'complien', 'cumplire', 'cumpliredes', 'cunplio', 'cumplir (s)a', 'conplien', 'cumple', 'conplidos', 'cumplieren', 'cumplieres', 'cumplades', 'cunpla', 'complir (s)an', 'complira', 'cunplian', 'conpliremos', 'cumpla', 'complid', 'complen', 'cumplieredes', 'cumpliran', 'cumpliras', 'conplir (s)a', 'cumplido', 'cumplan', 'cunpliran', 'cumpliestes', 'conplie', 'complieren', 'complir', 'cumplir', 'ayan complida', 'cumpliremos', 'complio', 'cunplira', 'auras complida', 'ouo complido', 'ayades cumplido', 'conpliran', 'cumplirien', 'complil(lo)', 'complida', 'complie', 'complirie', 'cunple', 'conprido', 'cunpliesse', 'cumpliere', 'complir se(a)', 'cumplieron', 'conplio', 'complire', 'cumplidos', 'cunplen', 'compliesse', 'conpliesen', 'complirien', 'cumplien', 'cumpliesse', 'cumplir (set)a', 'complides', 'complir (la) e', 'cunpliessen', 'conpliese', 'compliste', 'cumplist', 'cumplie'}
a = {'deffiende', 'deffendiemos', 'deffendido', 'defendiera', 'defiende', 'defendien', 'deffendieron', 'deffendries', 'defendiese', 'deffendiessemos', 'defende', 'deffendiesse', 'deffiendan', 'defendel(los)', 'deffendien', 'defendudas', 'deffendieran', 'defendre', 'deffendio', 'defendudo', 'defendieron', 'deffende', 'defendas', 'deffendudo', 'defender (l)an', 'defendrie', 'defiendan', 'deffendiera', 'deffenden', 'defiendas', 'deffendidas', 'deffendet', 'defiendo', 'defenduda', 'deffendi', 'defendemos', 'deffendre', 'auie defendido', 'defendra', 'deffendas', 'defendras', 'deffendra', 'deffendiendo', 'defenda', 'defenderedes', 'defendi', 'deffendiessen', 'auie deffendido', 'defendiesse', 'defendido he', 'defendo', 'defender', 'defendido', 'defendio', 'deffienden', 'defendiestes', 'defendie', 'deffiendas', 'deffendudas', 'defienden', 'defendet', 'deffiendo', 'defendiendo', 'deffendo', 'deffender', 'defendiere', 'defendades', 'deffienda', 'deffendie', 'deffendrien', 'deffendies', 'defendiessen', 'defendida', 'deffenda', 'deffenduda', 'deffendrie', 'defendudos', 'he defendido', 'as defendido', 'defendidos', 'deffendia', 'defienda'}
a = {'destruxeron', 'destruyst', 'destruyra', 'destruyr (ge la) an', 'destruxesse', 'destroyran', 'destruyr (se)a', 'destruyrie', 'estruyr', 'destroyesse', 'destroysse', 'estroyen', 'destruyas', 'destruen', 'destruydas', 'destruyria', 'destroydos', 'destruir (s)a', 'destruir', 'destruyo', 'estroyr', 'destroydo', 'destruir (les) as', 'destruido', 'destruyendo', 'destruyesse', 'destroyeran', 'destruyere', 'destruyr (los) an', 'destroyessen', 'destroiras', 'destroya', 'destruira', 'destruyan', 'estroyendo', 'destroyera', 'destroyr (los) e', 'destrue', 'estroyda', 'destroye', 'destruidos', 'destroyen', 'destruye', 'destruire', 'destroido', 'destruxo', 'destroyrie', 'destroyl(los)', 'estroydos', 'destroyda', 'destruyda', 'destruxieron', 'destroyr (s)a', 'destruyera', 'destruymos', 'destroyries', 'dextruxo', 'destroyr (uos) he', 'destruxiesse', 'destruyen', 'destruyr (los) a', 'destruyredes', 'destroyd', 'destruyre', 'destroyre', 'destruyessemos', 'destruyeran', 'destruyrien', 'destroyeron', 'destroil(la)', 'destruyd', 'destruyras', 'destruyr (nos) a', 'destroyendo', 'destroir', 'destruyamos', 'destroyra', 'auien destroidas', 'destruida', 'destroyrien', 'destruir (los) an', 'destruya', 'destruyr', 'destroyron', 'destroida', 'destruyessen', 'destroidos', 'destruydo', 'destruysse', 'destroyil(los)', 'destroidas', 'destroyssen', 'estroydo', 'destroy', 'destruyeron', 'destroydas', 'destruyran', 'destroyr', 'ouo destroidos', 'destruydos'}



b = set(re.sub(pattern, '', p) for p in a if p.count(' ')==0 and p.count('(')==0 and p.count(')')==0)
# for p in a:
#   print(p, re.sub(pattern, '', p))
print(b)

In [ ]:
#SuperIrregular
with open('lexemes.txt', 'a') as lex:
  lex.write(
f"""-lexeme
 lex: ser
 stem: s.
 gramm: verb
 paradigm: V_serS
""")
  lex.write(
f"""-lexeme
 lex: ser
 stem: e.|he.
 gramm: verb
 paradigm: V_serE
""")
  lex.write(
f"""-lexeme
 lex: ser
 stem: f.
 gramm: verb
 paradigm: V_serF
""")
  lex.write(
f"""-lexeme
 lex: ir
 stem: y.|i.|j.
 gramm: verb
 paradigm: V_irY
""")
  lex.write(
f"""-lexeme
 lex: ir
 stem: f.
 gramm: verb
 paradigm: V_irF
""")
  lex.write(
f"""-lexeme
 lex: ir
 stem: v.|u.
 gramm: verb
 paradigm: V_irV
""")
  lex.write(
f"""-lexeme
 lex: saber
 stem: se.
 gramm: verb,prs,ind,1sg
 paradigm: ZERO
""")


  lex.write(
f"""-lexeme
 lex: haber
 stem: he.|e.
 gramm: verb,prs,ind,1sg
 gloss: VERB.PRS.IND.1SG
 paradigm: ZERO
""")
  lex.write(
f"""-lexeme
 lex: haber
 stem: as.
 gramm: verb,prs,ind,2sg
 gloss: VERB.PRS.IND.2SG
 paradigm: ZERO
""")
  lex.write(
f"""-lexeme
 lex: haber
 stem: a.|ha.|ah.
 gramm: verb,prs,ind,3sg
 gloss: VERB.PRS.IND.3SG
 paradigm: ZERO
""")
  lex.write(
f"""-lexeme
 lex: haber
 stem: an.|han.
 gramm: verb,prs,ind,3pl
 gloss: VERB.PRS.IND.3PL
 paradigm: ZERO
""")

###Unidades pluriverbales

Verbos

In [ ]:
#como verbos mas no verbos
import re
dict_extr_R = df[
    (df['WORD'].str.strip().str.fullmatch(r".+[aie]r\b")) &
    ((df['POS'].str.strip() == 'Unidades pluriverbales:') &
    (
        (df['WORD'].isin({'adar', 'atacir', 'azar', 'azingar', 'bezoar', 'cantar', 'cuer', 'haber', 'juglar', 'lugar', 'mar', 'menester', 'mujer', 'pesar', 'poder', 'saber', 'ser'}))
        &~df.index.isin({5363, 7671, 7804, 8346, 8682, 9002})
    ))
]
with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_extr_R['WORD'], dict_extr_R['ADDITIONAL_INFO']):
    forms = set([re.sub(r'(.+)es', r'\1', form) if word[-2:]!='es' else re.sub(r'(.+ess?)es', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms else ''}
 gramm: noun
 paradigm: NOM_C
"""
    )

In [ ]:
import re
dict_extr_R = df[
    (df['WORD'].str.strip().str.fullmatch(r".+[^aie]r\b")) &
    (df['POS'].str.strip() == 'Unidades pluriverbales:') &
    (df['WORD']!='por')
]
with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_extr_R['WORD'], dict_extr_R['ADDITIONAL_INFO']):
    forms = set([re.sub(r'(.+)es', r'\1', form) if word[-2:]!='es' else re.sub(r'(.+ess?)es', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms else ''}
 gramm: noun
 paradigm: NOM_C
"""
    )

In [ ]:
dict_extr_R.drop('EXAMPLES', axis=1)

Nomina

In [ ]:
#in -o
import re
dict_noun_V = df[
    (df['WORD'].str.strip().str.fullmatch(r".+o\b")) &
    (df['POS'].str.strip() == 'Unidades pluriverbales:') &
    (df['WORD']!='no') & (df['WORD']!='como')
]
with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    if not isinstance(other, str):
          forms = set()
    else:
          forms = set(eval(other)['Formas atestiguadas:\xa0'])
    if any([re.fullmatch(r'.+as?', form) for form in forms]):
        word = re.sub(r'(.+)(o)', r'\1', word)
        if not isinstance(other, str):
          forms = set()
        else:
          forms = set([re.sub(r'(.+)([oa]s?)', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
        forms.discard(word)
        lex.write(
            f"""-lexeme
 lex: {word+'o'}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms and list(forms)[0] else ''}
 gramm: adj
 paradigm: ADJ
"""
        )
    else:
      if not isinstance(other, str):
          forms = set()
      else:
          forms = set([re.sub(r'(.+)s', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
      forms.discard(word)
      lex.write(
          f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms and list(forms)[0] else ''}
 gramm: noun
 paradigm: NOM
"""
      )

In [ ]:
# in -a
import re
dict_noun_V = df[
    (df['WORD'].str.strip().str.fullmatch(r".+a\b")) &
    (df['POS'].str.strip() == 'Unidades pluriverbales:') &
    (~df['WORD'].isin({'afuera', 'cada', 'contra', 'defuera'}))
]
with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    if 'Formas atestiguadas:\xa0' in eval(other):
      forms = set([re.sub(r'(.+)s', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    else:
      forms = set()
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: noun
 paradigm: NOM
"""
    )

In [ ]:
#other V
import re
dict_noun_V = df[
    (df['WORD'].str.strip().str.fullmatch(r".+[eiu]\b", na=False)) &
    (df['POS'].str.strip() == 'Unidades pluriverbales:') &
    (~df['WORD'].isin({'leve', 'su', 'tu', 'sobre', 'tarde', 'que', 'qui'})) &
    ((~df['WORD'].str.strip().str.contains(r"(ante|i?ente)$", regex=True, na=False))|
   ( df['WORD'].isin({'frente', 'gente', 'mente', 'occidente', 'oriente'})))
]
with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    forms = set([re.sub(r'(.+)s', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: noun
 paradigm: NOM
"""
    )

In [ ]:
dict_noun_V.drop('EXAMPLES', axis=1)

In [ ]:
#adj in V-o
import re
dict_noun_V = df[
    (df['POS'].str.strip() == 'Unidades pluriverbales:') &
    (df['WORD'].str.strip().str.fullmatch(r".+[eiu]\b", na=False)) &
    ((df['WORD'].isin({'leve','tarde'})) |
    (df['WORD'].str.strip().str.contains(r"(ante|i?ente)$", regex=True, na=False))) &
    (~df['WORD'].isin({'frente', 'gente', 'mente', 'occidente', 'oriente'}))
]
with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    word = re.sub(r'(.+)(e)', r'\1', word)
    if not isinstance(other, str):
      forms = set()
    else:
      forms = set([re.sub(r'(.+)(es?)', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word+'e'}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: adj
 paradigm: ADJ_e
"""
  )

In [ ]:
# in C-r
import re
dict_noun_V = df[
    (df['POS'].str.strip() == 'Unidades pluriverbales:') &
    (df['WORD'].str.strip().str.fullmatch(r".+[^aeiour]\b", na=False))&
    (~df['WORD'].isin({'antes', 'así', 'bien','ambos','azul','especial', 'demás', 'mientras', 'más', 'menos', 'hoy', 'pues', 'pos', 'oriental', 'occidental', 'tal'}))
]
with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    if 'Formas atestiguadas:\xa0' not in eval(other):
      forms = set()
    else:
      forms = set([re.sub(r'(.+)s', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: noun
 paradigm: NOM
"""
    )

In [ ]:
#rest of these
import re
dict_adj_C = df[
    (df['POS'].str.strip() == 'Unidades pluriverbales:') &
    (df['WORD'].str.strip().str.fullmatch(r".+[^aeiour]\b", na=False))&
    (df['WORD'].isin({'azul','especial', 'demás', 'oriental', 'occidental', 'tal'}))
]
with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_adj_C['WORD'], dict_adj_C['ADDITIONAL_INFO']):
    if not isinstance(other, str):
      forms = set()
    else:
      forms = set([re.sub(r'(.+)(es|(?<=a)s)\b', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
      forms.discard(word)
      lex.write(
  f"""-lexeme
  lex: {word}
  stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
  gramm: adj
  paradigm: ADJ_C
  """
    )
dict_noun_V = df[
    (df['POS'].str.strip() == 'Unidades pluriverbales:') &
    (df['WORD'].isin({'antes', 'así', 'bien', 'demás', 'mientras', 'más', 'menos', 'hoy', 'afuera', 'defuera', 'solamente', 'no', 'como'}))
]
with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    if 'Formas atestiguadas:\xa0' not in eval(other):
      forms = set()
    else:
      forms = set(eval(other)['Formas atestiguadas:\xa0'])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: adv
 paradigm: ZERO
"""
    )
  lex.write(
f"""-lexeme
 lex: ambos
 stem: amb.
 gramm: adj
 paradigm: ADJ
"""
    )
  lex.write(
f"""-lexeme
 lex: cada
 stem: cada.
 gramm: adj
 paradigm: ZERO
"""
    )

In [ ]:
dict_noun_V.drop('EXAMPLES', axis=1)

###ál

In [ ]:
#interj
import re
dict_noun_V = df[
    df['POS'].str.strip() == 'interj.'
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    forms = set(eval(other)['Formas atestiguadas:\xa0'])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: interj
 paradigm: ZERO
"""
    )

In [ ]:
#adv
import re
dict_noun_V = df[
    (df['POS'].str.strip() == 'adv.')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    forms = set(eval(other)['Formas atestiguadas:\xa0'])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: adv
 paradigm: ZERO
"""
    )

In [ ]:
#prep
import re
dict_noun_V = df[
    (df['POS'].str.strip() == 'prep.')|
    (
            (df['POS'].str.strip() == 'Unidades pluriverbales:') &
            (df['WORD'].isin({'por', 'contra', 'sobre', 'pos'}))
    )
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    forms = set(eval(other)['Formas atestiguadas:\xa0'])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: prep
 paradigm: ZERO
"""
    )

In [ ]:
#conj
import re
dict_noun_V = df[
    (df['POS'].str.strip() == 'conj.')|
    (
            (df['POS'].str.strip() == 'Unidades pluriverbales:') &
            (df['WORD'].isin({'pues', 'que', 'como'}))
    )
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    forms = set(eval(other)['Formas atestiguadas:\xa0'])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: conj
 paradigm: ZERO
"""
    )

In [ ]:
df['POS'].value_counts()

###Pronombres

In [ ]:
dfi = df[df['POS'] == 'contracc. ']

In [ ]:
dfi

In [ ]:
df[df['POS'] == 'pron. '].drop('EXAMPLES', axis=1)

In [ ]:
# pronomina en -e/o
import re
dict_adj_V = df[
    df.index.isin({596, 1062, 1064, 1065, 4669, 7054})
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_adj_V['WORD'], dict_adj_V['ADDITIONAL_INFO']):
    if not isinstance(other, str):
      forms = set()
    else:
      forms = set([re.sub(r'(.+)([eoa]s?)', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.add(re.sub(r'(.+)([eo])', r'\1', word))
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: pron
 paradigm: PRON
"""
    )

In [ ]:
# pronomina immutabilia
import re
dict_adj_V = df[
    (df.index.isin({419, 3053, 6978, 8158, 8174, 8445, 9086})) |
    (df['WORD']=='qui')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_adj_V['WORD'], dict_adj_V['ADDITIONAL_INFO']):
    if not isinstance(other, str):
      forms = set()
    else:
      forms = set(eval(other)['Formas atestiguadas:\xa0'])
    forms.add(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: pron
 paradigm: ZERO
"""
    )

In [ ]:
#pronomina personalia
with open('lexemes.txt', 'a') as lex:
      lex.write(
f"""-lexeme
 lex: yo
 stem: yo.|io.
 gramm: pron,1sg,nom
 gloss: PRON.1SG.NOM
 paradigm: ZERO
"""
      )
      lex.write(
f"""-lexeme
 lex: yo
 stem: m.
 gramm: pron,1sg
 gloss: PRON.1SG
 paradigm: PRON_yo
"""
      )
      lex.write(
f"""-lexeme
 lex: tú
 stem: t.
 gramm: pron,2sg
 gloss: PRON.2SG
 paradigm: PRON_tu
"""
      )
      lex.write(
f"""-lexeme
 lex: él
 stem: e.|é.
 gramm: pron,3
 paradigm: PRON_el1
"""
    )
      lex.write(
f"""-lexeme
 lex: él
 stem: l.
 gramm: pron,3
 gloss: PRON.3
 paradigm: PRON_el2
"""
      )
      lex.write(
f"""-lexeme
 lex: nos
 stem: n.
 gramm: pron,1pl
 gloss: PRON.1PL
 paradigm: PRON_nos
"""
      )
      lex.write(
f"""-lexeme
 lex: vos
 stem: v.|u.
 gramm: pron
 gramm: pron,2pl
 gloss: PRON.2PL
 paradigm: PRON_os
"""
      )
      lex.write(
f"""-lexeme
 lex: vos
 stem: os.
 gramm: pron
 gramm: pron,2pl,acc
 gloss: PRON.2PL.ACC
 paradigm: ZERO
"""
      )
      lex.write(
f"""-lexeme
 lex: vos
 stem: os.
 gramm: pron
 gramm: pron,2pl,dat
 gloss: PRON.2PL.DAT
 paradigm: ZERO
"""
      )
      lex.write(
f"""-lexeme
 lex: vos
 stem: os.
 gramm: pron
 gramm: pron,2pl,abl
 gloss: PRON.2PL.ABL
 paradigm: ZERO
"""
      )
      lex.write(
f"""-lexeme
 lex: se
 stem: s.
 gramm: pron
 gloss: PRON
 paradigm: PRON_se
"""
      )
      lex.write(
f"""-lexeme
 lex: él
 stem: se.|ge.|ie.|je.
 gramm: pron,3,dat,sg
 gloss: PRON.3.DAT.SG
 paradigm: ZERO
"""
      )
      lex.write(
f"""-lexeme
 lex: él
 stem: se.|ge.|ie.|je.
 gramm: pron,3,dat,pl
 gloss: PRON.3.DAT.PL
 paradigm: ZERO
"""
      )


###Articula

In [ ]:
with open('lexemes.txt', 'a') as lex:
      lex.write(
f"""-lexeme
 lex: el
 stem: l.
 gramm: art,def,masc,sg
 gloss: ART.DEF.M.SG
 paradigm: ART_el
"""
      )
      lex.write(
f"""-lexeme
 lex: el
 stem: el.|ell.
 gramm: art
 gloss: ART
 paradigm: ART_el
"""
      )
      lex.write(
f"""-lexeme
 lex: un
 stem: u.|v.
 gramm: art
 gloss: ART
 paradigm: ART_un
"""
      )

##Uniparser

In [ ]:
!pip install uniparser_morph

In [ ]:
from uniparser_morph import Analyzer
a = Analyzer()
a.lexFile = "lexemes.txt"
a.paradigmFile = "/content/project-753/paradigms.txt"
a.verbose = True
a.load_grammar()

In [ ]:
analyses = a.analyze_words(['el', "la", 'le', 'li'])
print(analyses)

In [ ]:
analyses = a.analyze_words('ovo')
# If you pass a single string, you will get a list of Wordform objects
# The analysis attributes are stored in its properties
# as string values, e.g.:
for ana in analyses:
        print(ana.wf, ana.lemma, ana.gramm, ana.gloss)
        print([ana.gramm, ana.gloss])

##Test

In [ ]:
import re
test = [(lambda x: {"word": x[0], "label": set(x[1].split(',')), "predicted": [set(prediction.gramm.split(',')) for prediction in a.analyze_words(x[0])]})(line.split(':')) for line in re.split(r"\n|\n\n", open('/content/project-753/test.txt', 'r').read().strip()) if line and line[0]!='-']
test[1]

In [ ]:
right, wrong, none = [],[],[]
for item in test:
  word, corr, preds = item.values()
  if not list(preds[0])[0]:
    none.append(item)
  elif any([corr==pred for pred in preds]):
    right.append(item)
  else:
    wrong.append(item)
num_right, num_wrong, num_none, num = len(right), len(wrong), len(none), len(test)
print(f'{num_right}/{num} of predictions are correct.\n{num_wrong}/{num} of predictions are wrong.\n{num_none}/{num} were not parsed.')

In [ ]:
wrong